<span style="color:blue">
    
---
# Analysis of data on the elections in Georgia 2020-11-03
    
---
    
</span>    

In [23]:
import os
import sys
import warnings
import datetime
import pdb
import xml
import re
import openpyxl
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotnine
from tqdm import tqdm
import collections
import selenium
now = datetime.datetime.now
now()

datetime.datetime(2020, 12, 23, 6, 1, 33, 263449)

<span style="color:blue">
    
---

## Results by County and VoteType

https://results.enr.clarityelections.com/GA/105369/web.264614/#/summary   
    or  
https://results.enr.clarityelections.com/GA/105369/web.264614/#/detail/10000    


another possible source  for the senate race
    
www.nbcnews.com/politics/2020-special-elections/georgia-senate-results
        
    
---

</span>

In [2]:

fpath = "./data/detailxml/detail.xml"
import xml.etree.ElementTree as et 
def node_msg(node, pref= '', tag=True, attribs=None):
    if attribs is None:
        attribs = []
    if isinstance(attribs, str):
        attribs = [attribs]
    msg = ""
    msg += "{0}{1}  tag: {2}".format(pref,node, node.tag)
    pref = pref+"  "    
    for at in attribs:
        atval = None
        if at in node.attrib.keys():
            atval = node.attrib[at]
        msg += "\n{0}{1} '{2}'".format(pref, at, atval)            
    return msg

In [3]:
CountyVoteTup = collections.namedtuple("CountyVoteTup",['CountyName', 'Votes'])
VoteTypeTup = collections.namedtuple("VoteTypeTup",    ['CountyName', 'Votes', 'VoteType'])    
ChoiceTup = collections.namedtuple("ChoiceTup",        ['CountyName', 'Votes', 'VoteType','Choice'])   
ContestTup = collections.namedtuple("ContestTup",      ['CountyName', 'Votes', 'VoteType','Choice', 'Contest'])  
def doCountyVote(node):
    county_name = node.attrib['name']
    votes = int(node.attrib['votes'])
    cv_tup = CountyVoteTup(county_name, votes)
    return cv_tup
def doVoteType(root_node, pref='', verbosity=0):
    if verbosity > 0:
        print(node_msg(root_node, pref=pref+'  ', attribs='text'))
    VoteTypeName = root_node.attrib['name']        
    counties = []
    votetype_tups = []
    for node in root_node:      
        if node.tag == "County":
            cv = doCountyVote(node)
            vt = VoteTypeTup(*cv, VoteTypeName)
            votetype_tups.append(vt)
            counties.append(cv)
    res = pd.DataFrame(counties)
    res.set_index("CountyName", inplace=True)
    res.rename(columns={"Votes":VoteTypeName}, inplace=True)
    return (res, votetype_tups)
def doChoice(root_node, pref='', verbosity=0):
    if verbosity > 0:
        print(node_msg(root_node, pref=pref+'  ', attribs='text'))   
    ChoiceText = root_node.attrib['text']         
    res = None
    choice_tups = []
    for node in root_node:
        if node.tag == "VoteType":
            df, vts = doVoteType(node)
            for vt in vts:
                choice = ChoiceTup(*vt, ChoiceText)
                choice_tups.append(choice)
            if res is None:
                res = df
            else:
                res = res.join(df, how="left")             
    res['Total'] = res.sum(axis=1)     
    res.columns = pd.MultiIndex.from_product([[ChoiceText], res.columns])   
    return res, choice_tups
def doContest(root_node, pref='', verbosity=0):
    if verbosity > 0:
        print(node_msg(root_node, pref=pref+'  ', attribs='text'))       
    res = None
    ContestText = root_node.attrib['text']     
    contest_tups = []
    for node in root_node:
        if node.tag == "Choice":
            choice, tups = doChoice(node)              
            for tup in tups:
                contest_tup = ContestTup(*tup, ContestText)
                contest_tups.append(contest_tup)
            if res is None:
                res = choice
            else:
                res = res.join(choice, how='left')
    return (res, contest_tups)

In [4]:
xtree = et.parse(fpath)
senate_races = []
xroot = xtree.getroot()
tups = []
contests = {}
for i, node in enumerate(tqdm(xroot)): 
    if node.tag == 'Contest':
        if 'text' in node.attrib.keys():
            if 'US Senate'  in node.attrib['text'] or \
                'President' in node.attrib['text'] :
                contest = node.attrib['text']
                res, contest_tups = doContest(node, pref='')
                tups.extend(contest_tups)
                print(contest)
                contests[contest] = res
contests.keys()
contestsdf = pd.DataFrame(tups)
contestsdf['CountyName'] = contestsdf['CountyName'].apply(lambda x: x.upper())
contestsdf.groupby(by="Contest").sum()

  2%|█▉                                                                                | 7/303 [00:00<00:31,  9.48it/s]

President of the United States
US Senate (Perdue)


100%|███████████████████████████████████████████████████████████████████████████████| 303/303 [00:01<00:00, 282.02it/s]

US Senate (Loeffler) - Special


,Votes
Contest,
President of the United States,4998482
US Senate (Loeffler) - Special,4914361
US Senate (Perdue),4952175


In [5]:
print(contestsdf['VoteType'].unique())
contestsdf.head(2)

['Election Day Votes' 'Absentee by Mail Votes' 'Advanced Voting Votes'
 'Provisional Votes']


,CountyName,Votes,VoteType,Choice,Contest
0,APPLING,1753,Election Day Votes,Donald J. Trump (I) (Rep),President of the United States
1,ATKINSON,716,Election Day Votes,Donald J. Trump (I) (Rep),President of the United States


In [6]:
grp_contests = contestsdf.groupby(by=['VoteType','Choice', 'Contest']).sum()
grp_contests.sort_values(by=['Contest', 'VoteType','Choice'],inplace=True)
grp_contests.reset_index(inplace=True)
grp_contests.head()

,VoteType,Choice,Contest,Votes
0,Absentee by Mail Votes,Donald J. Trump (I) (Rep),President of the United States,451157
1,Absentee by Mail Votes,Jo Jorgensen (Lib),President of the United States,16057
2,Absentee by Mail Votes,Joseph R. Biden (Dem),President of the United States,849729
3,Advanced Voting Votes,Donald J. Trump (I) (Rep),President of the United States,1419161
4,Advanced Voting Votes,Jo Jorgensen (Lib),President of the United States,25209


In [7]:
grp_contests.loc[grp_contests['Contest'].str.contains('President'),]
grp_contests.sort_values(by="Votes", inplace=True)
grp_contests.loc[grp_contests['Contest'].str.contains('Loeffler'),].tail(10)

,VoteType,Choice,Contest,Votes
37,Advanced Voting Votes,Deborah Jackson (Dem),US Senate (Loeffler) - Special,173820
19,Absentee by Mail Votes,Doug Collins (Rep),US Senate (Loeffler) - Special,185300
68,Election Day Votes,Raphael Warnock (Dem),US Senate (Loeffler) - Special,205110
59,Election Day Votes,Doug Collins (Rep),US Senate (Loeffler) - Special,211902
25,Absentee by Mail Votes,Kelly Loeffler (I) (Rep),US Senate (Loeffler) - Special,249090
65,Election Day Votes,Kelly Loeffler (I) (Rep),US Senate (Loeffler) - Special,298209
39,Advanced Voting Votes,Doug Collins (Rep),US Senate (Loeffler) - Special,581871
28,Absentee by Mail Votes,Raphael Warnock (Dem),US Senate (Loeffler) - Special,601522
45,Advanced Voting Votes,Kelly Loeffler (I) (Rep),US Senate (Loeffler) - Special,724577
48,Advanced Voting Votes,Raphael Warnock (Dem),US Senate (Loeffler) - Special,807005


In [11]:
presdf = contestsdf[contestsdf['Contest'].str.contains("President")]
presByCounty = pres[["CountyName", "Votes"]].groupby("CountyName").sum()
presByCounty.sort_values(by="Votes", ascending=False, inplace=True)
presByCounty.head(4)

,Votes
CountyName,
FULTON,524659
GWINNETT,413865
COBB,393746
DEKALB,370804


In [12]:
loefflerdf = contestsdf[contestsdf['Contest'].str.contains("Loeffler")]
purduedf = contestsdf[contestsdf['Contest'].str.contains("Loeffler")]
if True:
    gdf = presdf[['VoteType', 'Votes', 'Contest']].groupby('VoteType').sum() 
    gdf["Pct"] = 100*np.round(gdf / gdf.sum(),3)
    print("Votes counted by Vote Type")
    print(gdf)
    print("------")
    print("Total votes counted (millions): {0}".format(gdf["Votes"].sum()/10.0**6))

Votes counted by Vote Type
                          Votes   Pct
VoteType                             
Absentee by Mail Votes  1316943  26.3
Advanced Voting Votes   2694879  53.9
Election Day Votes       975540  19.5
Provisional Votes         11120   0.2
------
Total votes counted (millions): 4.998482


In [13]:
votesByCounty = contestsdf[["CountyName", "Votes"]].groupby("CountyName").sum()
votesByCounty.sort_values(by="Votes", ascending=False, inplace=True)
votesByCounty.head(4)

,Votes
CountyName,
FULTON,1563044
GWINNETT,1227786
COBB,1173481
DEKALB,1105580


<span style="color:blue">
    
---
    
# Registered voters by County, Age, Race
    
https://sos.ga.gov/index.php/Elections/voter_registration_statistics
    
---
    
</span>

In [14]:
if True:
    dpath = "./data/"
    voterfile = "Active_Voters_by_Race_Gender_and_Age_as_of_November_1_2020.xlsx"
    voterpath = os.path.join(dpath, voterfile)
    if not os.path.isdir(dpath):
        warnings.warn("{0} is not a directory".format(dpath))
    if not os.path.isfile(voterpath):
        warnings.warn("No file {0} in dir {1}".format(voterfile, dpath))
    with open(voterpath) as vp:
        racedf = pd.read_excel(voterpath, nrows=7, engine='openpyxl').iloc[:,:2]
        racedf.columns = ["RaceCode", "RaceDescription"]
        racedf["RaceCode"] = racedf["RaceCode"].apply(lambda x: x[:-2])
        racedf.set_index("RaceCode", inplace=True)
        race = racedf.squeeze()
        race_dict = race.to_dict()
        print(race_dict)

{'AI': 'AMERICAN INDIAN OR ALASKAN NATIVE', 'AP': 'ASIAN OR PACIFIC ISLANDER', 'BH': 'BLACK NOT OF HISPANIC ORIGIN', 'HP': 'HISPANIC', 'WH': 'WHITE NOT OF HISPANIC ORIGIN', 'OT': 'OTHER', 'UK': 'UNKNOWN'}


In [15]:

votersdf = pd.read_excel(voterpath, skiprows=8, engine='openpyxl')
if votersdf.columns[-1] != "TOTAL VOTERS":
    votersdf = votersdf.iloc[:,:-1]
colnames = [re.sub("\s+", "-", c.strip()) for c in votersdf.columns]
votersdf.columns = colnames

votersdf.drop('COUNTY-ID', axis=1, inplace=True)    
votersdf = votersdf.loc[votersdf["COUNTY-NAME"].notna()]
colname_parts = [c.split("-") for c in votersdf.columns]
newnames = []

for parts in colname_parts:
    newparts = parts
    if parts[0]  in race_dict.keys():
        newparts = [race_dict[parts[0]].split(" ")[0]] + parts[1:]
    newnames.append("-".join(newparts))
votersdf.columns = newnames
votersdf.rename(columns={"COUNTY-NAME":"CountyName", 
                        'TOTAL-VOTERS':'TotalVoters'}, inplace=True)
votersdf['CountyName'] = votersdf['CountyName'].apply(lambda x: x.upper())
print("{0} rows {1} cols".format(*votersdf.shape))
votersdf.head(2)

1594 rows 24 cols


,CountyName,VOTER-AGE-RANGE,AMERICAN-MALE-VOTERS,AMERICAN-FEMALE-VOTERS,AMERICAN-UNKNOWN-VOTERS,ASIAN-MALE-VOTERS,ASIAN-FEMALE-VOTERS,ASIAN-UNKNOWN-VOTERS,BLACK-MALE-VOTERS,BLACK-FEMALE-VOTERS,...,WHITE-MALE-VOTERS,WHITE-FEMALE-VOTERS,WHITE-UNKNOWN-VOTERS,OTHER-MALE-VOTERS,OTHER-FEMALE-VOTERS,OTHER-UNKNOWN-VOTERS,UNKNOWN-MALE-VOTERS,UNKNOWN-FEMALE-VOTERS,UNKNOWN-UNKNOWN-VOTERS,TotalVoters
0,APPLING,18-24,2,1,0,3,5,0,120,110,...,436,429,1,5,19,0,95,60,2,1374
1,APPLING,25-29,0,0,0,4,1,0,81,89,...,282,298,1,3,7,0,38,30,0,866


In [16]:
gcols = ["CountyName"]
votersByCounty = votersdf[["CountyName", 'TotalVoters' ]].groupby(by=gcols).sum()
votersByCounty.sort_values(by="TotalVoters", ascending=False, inplace=True)
votersByCounty.dtypes

TotalVoters    int64
dtype: object

<span style="color:blue">
    
---
    
# Combining Voters and Votes by County
    
---
    
</span>

In [17]:
print(presByCounty.index.values[:10])
print(votersByCounty.index.values[:10])


['FULTON' 'GWINNETT' 'COBB' 'DEKALB' 'CHEROKEE' 'CHATHAM' 'FORSYTH'
 'HENRY' 'CLAYTON' 'HALL']
['FULTON' 'GWINNETT' 'DEKALB' 'COBB' 'CHATHAM' 'CLAYTON' 'CHEROKEE'
 'HENRY' 'FORSYTH' 'RICHMOND']


In [18]:
presCombo = presByCounty.join(votersByCounty)
presCombo["PctVoted"] = np.round(100*presCombo['Votes'] / presCombo['TotalVoters'],3)
presCombo.sort_values(by="Votes", inplace=True)
presCombo.tail(10)

,Votes,TotalVoters,PctVoted
CountyName,,,
HALL,90523,128535,70.427
CLAYTON,112344,194338,57.809
HENRY,122742,172241,71.262
FORSYTH,129305,164279,78.711
CHATHAM,133420,202999,65.724
CHEROKEE,144830,190605,75.984
DEKALB,370804,547802,67.689
COBB,393746,537659,73.233
GWINNETT,413865,582917,70.999


<span style="color:blue">
    
---
    
# Voter Registration
    
https://elections.sos.ga.gov/Elections/voterhistory.do

---
    
</span>

In [19]:
ElectionType = {
'001':'GENERAL PRIMARY',
'002':'GENERAL PRIMARY RUN-OFF',
'003':'GENERAL',
'004':'GENERAL ELECTION RUNOFF',
'005':'SPECIAL ELECTION',
'006':'SPECIAL RUNOFF',
'007':'NON- PARTISAN',
'008':'SPECIAL/NON-PARTISAN',
'009':'RECALL',
'010':'PPP'
}
#SPECIAL PRIMARY RUNOFF
#SPECIAL PRIMARY

In [20]:
dpath = "./data/2020"
files = os.listdir(dpath)
fname = "2020.TXT"
if fname not in files:
    warnings.warn("{0} not found".format(fname))
if not os.path.isfile(os.path.join(dpath, fname)):
    warnings.warn("{0} not a file".format(fname))
fpath = os.path.join(dpath, fname)
print("Detailed registerd voter data for 2020 in the file '{0}' in the directory {1}".format(fname, dpath))

FileNotFoundError: [WinError 3] The system cannot find the path specified: './data/2020'

## Pandas has a method for reading in fixed width data

panda.read_fwf


In [80]:
dpath = "./data/2020"
files = os.listdir(dpath)
fname = "2020.TXT"
if fname not in files:
    warnings.warn("{0} not found".format(fname))
if not os.path.isfile(os.path.join(dpath, fname)):
    warnings.warn("{0} not a file".format(fname))
    
with open(os.path.join(dpath, fname), mode='r') as fp:
    txt = fp.read()
voter_lines = txt.split('\n')
minlen = min([len(l) for l in voter_lines])
maxlen = max([len(l) for l in voter_lines])
print("{0} had {1} lines, minlen {2}, maxlen {3}".format(fname, len(voter_lines), minlen, maxlen))

2020.TXT had 8416980 lines, minlen 0, maxlen 27


In [81]:
def parse_voter_lines(lines=None):
    from collections import namedtuple
    from collections import OrderedDict
    field_lens = {'county_num':3,
                'registration_num':8,
                'election_date':8,
                'election_type': 3,
                'party': 2,
                'absentee':1,
                'provisional':1,
                'supplemental':1}
    fields = ['county_num',
                'registration_num',
                'election_date',
                'election_type',
                'party',
                'absentee',
                'provisional',
                'supplemental']
    #print(field_lens.keys())
    VoterRec = namedtuple("VoterRec", fields )
    expected_len = sum([field_lens[k] for k in field_lens.keys()])
    tups = []
    #pdb.set_trace()
    for i, line in enumerate(tqdm(lines)):
        if len(line) != expected_len:
            msg = "{0} len={1}, line={2}".format(i, len(line), line)
            continue
        start = 0
        data = {}
        for f in fields:
            flen = field_lens[f]
            delem = line[start:(start+flen)]
            data[f] = delem
            start += flen
        vr = VoterRec(**data)
        tups .append(vr)
    df = pd.DataFrame(tups)
    return df, tups
regdf, tups = parse_voter_lines(lines=voter_lines)
print(tups[:2])


100%|████████████████████████████████████████████████████████████████████| 8416980/8416980 [00:52<00:00, 158840.24it/s]


[VoterRec(county_num='067', registration_num='02788924', election_date='20200324', election_type='005', party='  ', absentee='N', provisional='N', supplemental='N'), VoterRec(county_num='067', registration_num='05761105', election_date='20200324', election_type='005', party='  ', absentee='N', provisional='N', supplemental='N')]


In [83]:
print(regdf.head(2))
etype = '003'
gendf = regdf.loc[regdf['election_type'] == etype]
print("{0} rows for election type = {1}".format(gendf.shape[0], etype))
gendf.head(2)

  county_num registration_num election_date election_type party absentee  \
0        067         02788924      20200324           005              N   
1        067         05761105      20200324           005              N   

  provisional supplemental  
0           N            N  
1           N            N  
5002773 rows for election type = 003


,county_num,registration_num,election_date,election_type,party,absentee,provisional,supplemental
5419,019,00591608,20201103,003,,Y,N,N
5420,027,00430264,20201103,003,,Y,N,N


In [88]:
inames = []
for name in ['absentee', 'provisional', 'supplemental']:
    iname = name+'_int'
    inames.append(iname)
    gendf[iname] = [int(val=='Y') for val in gendf[name].values]

xdf[inames].sum()
gendf.head(2)

<ipython-input-88-9f39c7126d79>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


absentee_int        4018918
provisional_int        6474
supplemental_int        138
dtype: int64

In [78]:
print("rows in gendf.shape)
gendf['typesum'] = gendf[inames].sum(axis=1)
gendf.head()
print(gendf.shape[0] - gendf.loc[gendf['typesum'] != 1].shape[0])
print(gendf.shape)
#print(gendf[inames].sum().sum())

SyntaxError: EOL while scanning string literal (<ipython-input-78-4868ae74eb2a>, line 1)

In [ ]:
allcols = ['county_num', 'election_date', 'party', 'absentee', 'provisional', 'supplemental']
gcols = ['county_num', 'election_date', 'party']           
gdf = gendf[allcols].groupby(by=gcols)
gdf

In [ ]:
gendf['election_date'].unique()

In [ ]:
fpath = os.path.join('.',"data")
fname = "County_Number_List.pdf"
# Creating a pdf file object.
pdf = open(fpath, mode="rb")
 
# Creating pdf reader object.
pdf_reader = PyPDF2.PdfFileReader(pdf)
 
# Checking total number of pages in a pdf file.
print("Total number of Pages:", pdf_reader.numPages)
 
# Creating a page object.
page = pdf_reader.getPage(200)
 
# Extract data from a specific page number.
print(page.extractText())
 
# Closing the object.
pdf.close()

In [ ]:
# Election Types
election_types = {
    '001':'GENERAL PRIMARY',
    '002':'GENERAL PRIMARY RUN-OFF',
    '003':'GENERAL',
    '004':'GENERAL ELECTION RUNOFF',
    '005':'SPECIAL ELECTION',
    '006':'SPECIAL RUNOFF',
    '007':'NON- PARTISAN',
    '008':'SPECIAL/NON-PARTISAN',
    '009': ' RECALL',
    '010': 'PPP'
    #SPECIAL PRIMARY RUNOFF
    #SPECIAL PRIMARY
    }

In [ ]:
gcols = ['election_date', 'election_type']
df.groupby(gcols).count()

In [ ]:
fpath = os.path.join(os.getcwd(),"data")
fname = "County_Number_List.pdf"
if not os.path.isfile(os.path.join(fpath, fname)):
    if not os.path.isdir(fpath):
        warnings.warn("{0} not a directory".format(fpath))
    elif fname not in os.listdir(fpath):
        warnings.warn("{0} not a file in {1}".format(fname, fpath))
table = tabula.read_pdf(fpath,pages=1)
table[0]